## DATA SET

In [11]:
import pandas as pd

#-----importing Data Set
data = pd.read_csv("top10K-TMDB-movies.csv")
#data.head(10)

#-----checking null values and working on it
print(data['overview'].isnull().sum())
data[data['overview'].isna()]
data.isnull().sum()
data[data['genre'].isna()]



13


,id,title,genre,original_language,overview,popularity,release_date,vote_average,vote_count
3361,50472,Anplagghed al cinema,NaN,it,"A queue at the ATM machine, a displaced family...",4.420,2006-11-26,7.0,313
7821,43211,7 Kilos in 7 Days,NaN,it,Two not very clever young doctors open a fitne...,5.885,1986-02-02,6.0,212
8518,57114,"Amore, bugie e calcetto",NaN,en,NaN,4.709,2008-04-04,5.8,200


## Feature Selection 

In [ ]:
movies = data[['id','title','genre','overview']]

#Adding genre and overview into one "tags"
movies['tags'] = movies['genre'] + movies['overview']
movies

#Removing genre and overview columns
new_data = movies.drop(columns = ['overview','genre'])

C:\Users\Niranjan\AppData\Local\Temp\ipykernel_10268\1296373185.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['tags'] = movies['genre'] + movies['overview']


In [31]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=10000,stop_words = 'english')
CountVectorizer(max_features=100000,stop_words ='english')
vector = cv.fit_transform(new_data['tags'].values.astype('U')).toarray()
vector.shape

(10000, 10000)

## Cosine Similarity technique

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)

#Accessing similar movies 
distance = sorted(list(enumerate(similarity[2])), reverse = True, key = lambda vector:vector[1])
for i in distance[0:5]:
    print(new_data.iloc[i[0]].title)


The Godfather
The Godfather: Part II
Felon
House of Gucci
Gotti


## Making Function 

In [40]:
def recommend(movies):
    index = new_data[new_data['title']==movies].index[0]

    distance = sorted(list(enumerate(similarity[index])), reverse = True, key = lambda vector:vector[1])
    for i in distance[0:5]:
        print(new_data.iloc[i[0]].title)

recommend("Iron Man")

Iron Man
Mazinger Z: Infinity
Justice League Dark
Iron Man 3
The Colony


## Saving the model in pickle file

In [46]:
import pickle
pickle.dump(new_data, open('movies_list.pkl','wb'))
pickle.dump(similarity, open('similarity.pkl','wb'))

In [45]:
pickle.load(open('movies_list.pkl','rb'))

,id,title,tags
0,278,The Shawshank Redemption,"Drama,CrimeFramed in the 1940s for the double ..."
1,19404,Dilwale Dulhania Le Jayenge,"Comedy,Drama,RomanceRaj is a rich, carefree, h..."
2,238,The Godfather,"Drama,CrimeSpanning the years 1945 to 1955, a ..."
3,424,Schindler's List,"Drama,History,WarThe true story of how busines..."
4,240,The Godfather: Part II,"Drama,CrimeIn the continuing saga of the Corle..."
...,...,...,...
9995,10196,The Last Airbender,"Action,Adventure,FantasyThe story follows the ..."
9996,331446,Sharknado 3: Oh Hell No!,"Action,TV Movie,Science Fiction,Comedy,Adventu..."
9997,13995,Captain America,"Action,Science Fiction,WarDuring World War II,..."
9998,2312,In the Name of the King: A Dungeon Siege Tale,"Adventure,Fantasy,Action,DramaA man named Farm..."
